# Team Benchmark

**Authors:** Marissa Nicole Esteban, Gabe Krishnadasan, Diana Montoya-Herrera, Gabe Seidl, Madeleine Woo

**Date:** 2/13/2024

In [4]:
from seebuoy import NDBC
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm, skew, probplot
from scipy.special import boxcox1p
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

ndbc = NDBC()

# Information on NDBC's ~1800 buoys and gliders
wave_df = ndbc.stations()

# list all available data for all buoys
df_data = ndbc.available_data()

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
df_buoys = ndbc.stations()
m = df_buoys["closest_state"] == "New York"
df_ny = df_buoys[m]

df_available = ndbc.available_data(dataset="all")

# subset down to ny stations
m = df_available["station_id"].isin(df_ny["station_id"])
df_ny_avail = df_available[m] # using the mask

piv_ny = pd.pivot_table(
    df_ny_avail, 
    index="station_id", 
    columns="dataset", 
    aggfunc=len, 
    values="file_name"
)

ny_station_ids = piv_ny.index.tolist()
print("Number of stations around NY: ", len(ny_station_ids))
print(ny_station_ids)

In [ ]:
# creating a df for the combined data for all buoys around NY
ny_buoys_standard = pd.DataFrame()

for station_id in ny_station_ids:

    # get the standard data for a singular buoy
    df_station = ndbc.get_data(station_id, dataset="standard")
    
    # Add a column for station ID
    df_station['station_id'] = station_id
    
    # Concatenate the current station's data to the combined DataFrame
    ny_buoys_standard = pd.concat([ny_buoys_standard, df_station], ignore_index=False)


print(ny_buoys_standard.shape)
print(ny_buoys_standard['station_id'].nunique())
ny_buoys_standard.head(2)

In [ ]:
ny_buoy = ny_buoys_standard

### Handling Missing Data

In [ ]:
# dropping cols where there is 100% NA
ny_buoy.dropna(axis=1, how='all', inplace=True)

# dropping rows where average_period is null
ny_buoy.dropna(subset=['average_period'], inplace=True)

# dropping rows wehre wave_height is null
ny_buoy.dropna(subset=['wave_height'], inplace=True)

In [ ]:
from sklearn.impute import SimpleImputer

# Replace missing data with mode
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(ny_buoy)
ny_buoy_mode = pd.DataFrame(imputer.transform(ny_buoy), columns=ny_buoy.columns)

# Replace missing data with mean
imputer = SimpleImputer(strategy='mean')
imputer.fit(ny_buoy)
ny_buoy_mean = pd.DataFrame(imputer.transform(ny_buoy), columns=ny_buoy.columns)

# Interpolate missing values using spline interpolation
ny_buoy_interpolated = ny_buoy.interpolate(method='spline', order=2)

### Handling Outliers

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = ny_buoy['wind_gust'], y = ny_buoy['wave_height'])
plt.ylabel('Significant Wave Height', fontsize=13)
plt.xlabel('Wind Gust', fontsize=13)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = ny_buoy['mean_wave_direction'], y = ny_buoy['average_period'],)
plt.ylabel('Average Period', fontsize=13)
plt.xlabel('Mean Wave Direction', fontsize=13)
plt.show()

In [ ]:
# Remove non finite values
ny_buoy_mode = ny_buoy_mode[np.isfinite(ny_buoy_mode['wave_height'])]
ny_buoy_mean = ny_buoy_mean[np.isfinite(ny_buoy_mean['wave_height'])]
ny_buoy_interpolated = ny_buoy_interpolated[np.isfinite(ny_buoy_interpolated['wave_height'])]

In [ ]:
# We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
ny_buoy_mode["wave_height"] = np.log1p(ny_buoy_mode["wave_height"])
ny_buoy_mean["wave_height"] = np.log1p(ny_buoy_mean["wave_height"])
ny_buoy_interpolated["wave_height"] = np.log1p(ny_buoy_interpolated["wave_height"])


# Check the new distribution 
sns.distplot(ny_buoy_mode['wave_height'] , fit=norm)

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(ny_buoy_mode['wave_height'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

# Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Wave height distribution')

# Get also the QQ-plot
fig = plt.figure()
res = probplot(ny_buoy_mode['wave_height'], plot=plt)
plt.show()

In [ ]:
# We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
ny_buoy_mode["average_period"] = np.log1p(ny_buoy_mode["average_period"])
ny_buoy_mean["average_period"] = np.log1p(ny_buoy_mean["average_period"])
ny_buoy_interpolated["average_period"] = np.log1p(ny_buoy_interpolated["average_period"])

# Check the new distribution 
sns.distplot(ny_buoy_mode['average_period'] , fit=norm)

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(ny_buoy_mode['average_period'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

# Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Wave height distribution')

# Get also the QQ-plot
fig = plt.figure()
res = probplot(ny_buoy_mode['average_period'], plot=plt)
plt.show()

### Data Analysis and Visualization

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# scatterplot
sns.set()
cols = ['wind_speed', 'wind_gust', 'wave_height', 
        'dominant_period', 'average_period', 'mean_wave_direction', 
        'pressure', 'water_temp',
        'pressure_tendency']
sns.pairplot(ny_buoy_mode[cols], size = 2.5)
plt.show()

In [ ]:
# Correlation map to see how conditions are correlated with wave height and average period
corrmat = ny_buoy_mode.corr()
f, ax = plt.subplots(figsize=(15, 12))
sns.heatmap(corrmat, vmax=.8, square=True)

# Data Analytics
## Using algorithms on different imputations
* [ny_buoy_mode](#mode)
* [ny_buoy_mean](#mean)
* [ny_buoy_interpolated](#interpolate)

In [ ]:
import sys

!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install lightgbm

from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
import lightgbm as lgb

In [ ]:
train_df = ny_buoy[ny_buoy_mode.columns.difference(['wave_height', 'average_period'])]
train_df.head()

In [ ]:
# Validation function for wave height
n_folds = 5

def rmse_cv(model,n_folds):
    kf=KFold(n_splits=n_folds)
    rmse = np.sqrt(-cross_val_score(model, train_df, ny_buoy_mode.wave_height, scoring="neg_mean_squared_error", cv = kf))
    return rmse

lr_w_int = LinearRegression()
lr_no_int = LinearRegression(fit_intercept=False)

neigh = KNeighborsRegressor(n_neighbors=10)
rf = RandomForestRegressor(n_estimators=100)
dt = DecisionTreeRegressor(max_depth = 10)
model_xgb = xgb.XGBRegressor(max_depth=5, n_estimators=1000, learning_rate=0.01)
model_lgb = lgb.LGBMRegressor(learning_rate=0.01, max_depth=5, n_estimators=1000)

score_dt = rmse_cv(dt,n_folds)
print("Decision Tree Regression score: {:.4f} ({:.4f})\n".format(score_dt.mean(), score_dt.std()))

score_rf = rmse_cv(rf,n_folds)
print("Random Forest Regression score: {:.4f} ({:.4f})\n".format(score_rf.mean(), score_rf.std()))

score_xg = rmse_cv(model_xgb,n_folds)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score_xg.mean(), score_xg.std()))

score_lgbm = rmse_cv(model_lgb,n_folds)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score_lgbm.mean(), score_lgbm.std()))

In [ ]:
model = model_xgb.fit(train_df, ny_buoy_mode.wave_height) #fit model on entire dataset to get variable importance since we fit it on each fold
feature_important = model.get_booster().get_score(importance_type='weight')

keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data[:20].plot(kind='barh', figsize = (20,10)).invert_yaxis(); ## plot top 20 features
plt.xlabel("Feature Importance",fontsize=20)
plt.ylabel("Feature Name",fontsize=20)
plt.title("Feature Importance Plot",fontsize=20)
plt.show()

#### Repeat Training Models, but for Average Period

In [ ]:
# Validation function for wave height
n_folds = 5

def rmse_cv_period(model,n_folds):
    kf=KFold(n_splits=n_folds)
    rmse_period = np.sqrt(-cross_val_score(model, train_df, ny_buoy_mode.average_period, scoring="neg_mean_squared_error", cv = kf))
    return rmse_period

score_dt = rmse_cv_period(dt,n_folds)
print("Decision Tree Regression score: {:.4f} ({:.4f})\n".format(score_dt.mean(), score_dt.std()))

score_rf = rmse_cv_period(rf,n_folds)
print("Random Forest Regression score: {:.4f} ({:.4f})\n".format(score_rf.mean(), score_rf.std()))

score_xg = rmse_cv_period(model_xgb,n_folds)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score_xg.mean(), score_xg.std()))

score_lgbm = rmse_cv_period(model_lgb,n_folds)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score_lgbm.mean(), score_lgbm.std()))

In [ ]:
model = model_lgb.fit(train_df, ny_buoy_mode.average_period) #fit model on entire dataset to get variable importance since we fit it on each fold
lgb.plot_importance(model, importance_type="gain", figsize=(7,6), title="LightGBM Feature Importance (Gain)")

In [ ]:
# Plot feature importance using Split
lgb.plot_importance(model, importance_type="split", figsize=(7,6), title="LightGBM Feature Importance (Split)")
plt.show()

### Imputed on mean <a class="anchor" id="mean"></a>

In [ ]:
train_df = ny_buoy_mean[ny_buoy_mean.columns.difference(['wave_height', 'average_period'])]
train_df.head()

# Validation function for wave height
n_folds = 5

def rmse_cv(model,n_folds):
    kf=KFold(n_splits=n_folds)
    rmse = np.sqrt(-cross_val_score(model, train_df, ny_buoy_mean.wave_height, scoring="neg_mean_squared_error", cv = kf))
    return rmse

lr_w_int = LinearRegression()
lr_no_int = LinearRegression(fit_intercept=False)
neigh = KNeighborsRegressor(n_neighbors=10)
rf = RandomForestRegressor(n_estimators=100)
dt = DecisionTreeRegressor(max_depth = 10)
model_xgb = xgb.XGBRegressor(max_depth=5, n_estimators=1000, learning_rate=0.01)
model_lgb = lgb.LGBMRegressor(learning_rate=0.01, max_depth=5, n_estimators=1000)

score_dt = rmse_cv(dt,n_folds)
print("Decision Tree Regression score: {:.4f} ({:.4f})\n".format(score_dt.mean(), score_dt.std()))

score_rf = rmse_cv(rf,n_folds)
print("Random Forest Regression score: {:.4f} ({:.4f})\n".format(score_rf.mean(), score_rf.std()))

score_xg = rmse_cv(model_xgb,n_folds)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score_xg.mean(), score_xg.std()))

score_lgbm = rmse_cv(model_lgb,n_folds)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score_lgbm.mean(), score_lgbm.std()))

In [ ]:
model = model_xgb.fit(train_df, ny_buoy_mean.wave_height) #fit model on entire dataset to get variable importance since we fit it on each fold
feature_important = model.get_booster().get_score(importance_type='weight')

keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data[:20].plot(kind='barh', figsize = (20,10)).invert_yaxis(); ## plot top 20 features
plt.xlabel("Feature Importance",fontsize=20)
plt.ylabel("Feature Name",fontsize=20)
plt.title("Feature Importance Plot",fontsize=20)
plt.show()

#### Repeat Training Models, but for Average Period

In [ ]:
def rmse_cv(model,n_folds):
    kf=KFold(n_splits=n_folds)
    rmse = np.sqrt(-cross_val_score(model, train_df, ny_buoy_mean.average_period, scoring="neg_mean_squared_error", cv = kf))
    return rmse

lr_w_int = LinearRegression()
lr_no_int = LinearRegression(fit_intercept=False)
neigh = KNeighborsRegressor(n_neighbors=10)
rf = RandomForestRegressor(n_estimators=100)
dt = DecisionTreeRegressor(max_depth = 10)
model_xgb = xgb.XGBRegressor(max_depth=5, n_estimators=1000, learning_rate=0.01)
model_lgb = lgb.LGBMRegressor(learning_rate=0.01, max_depth=5, n_estimators=1000)

score_dt = rmse_cv(dt,n_folds)
print("Decision Tree Regression score: {:.4f} ({:.4f})\n".format(score_dt.mean(), score_dt.std()))

score_rf = rmse_cv(rf,n_folds)
print("Random Forest Regression score: {:.4f} ({:.4f})\n".format(score_rf.mean(), score_rf.std()))

score_xg = rmse_cv(model_xgb,n_folds)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score_xg.mean(), score_xg.std()))

score_lgbm = rmse_cv(model_lgb,n_folds)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score_lgbm.mean(), score_lgbm.std()))

### Imputed on interpolation <a class="anchor" id="interpolate"></a>

In [ ]:
train_df = ny_buoy_interpolated[ny_buoy_interpolated.columns.difference(['wave_height', 'average_period'])]
train_df.head()

# Validation function for wave height
n_folds = 5

def rmse_cv(model,n_folds):
    kf=KFold(n_splits=n_folds)
    rmse = np.sqrt(-cross_val_score(model, train_df, ny_buoy_interpolated.wave_height, scoring="neg_mean_squared_error", cv = kf))
    return rmse

lr_w_int = LinearRegression()
lr_no_int = LinearRegression(fit_intercept=False)
neigh = KNeighborsRegressor(n_neighbors=10)
rf = RandomForestRegressor(n_estimators=100)
dt = DecisionTreeRegressor(max_depth = 10)
model_xgb = xgb.XGBRegressor(max_depth=5, n_estimators=1000, learning_rate=0.01)
model_lgb = lgb.LGBMRegressor(learning_rate=0.01, max_depth=5, n_estimators=1000)

score_dt = rmse_cv(dt,n_folds)
print("Decision Tree Regression score: {:.4f} ({:.4f})\n".format(score_dt.mean(), score_dt.std()))

score_rf = rmse_cv(rf,n_folds)
print("Random Forest Regression score: {:.4f} ({:.4f})\n".format(score_rf.mean(), score_rf.std()))

score_xg = rmse_cv(model_xgb,n_folds)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score_xg.mean(), score_xg.std()))

score_lgbm = rmse_cv(model_lgb,n_folds)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score_lgbm.mean(), score_lgbm.std()))

In [ ]:
model = model_xgb.fit(train_df, ny_buoy_interpolated.wave_height) #fit model on entire dataset to get variable importance since we fit it on each fold
feature_important = model.get_booster().get_score(importance_type='weight')

keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data[:20].plot(kind='barh', figsize = (20,10)).invert_yaxis(); ## plot top 20 features
plt.xlabel("Feature Importance",fontsize=20)
plt.ylabel("Feature Name",fontsize=20)
plt.title("Feature Importance Plot",fontsize=20)
plt.show()

#### Repeat Training Models, but for Average Period (interpolate)

In [ ]:
def rmse_cv(model,n_folds):
    kf=KFold(n_splits=n_folds)
    rmse = np.sqrt(-cross_val_score(model, train_df, ny_buoy_interpolated.average_period, scoring="neg_mean_squared_error", cv = kf))
    return rmse

lr_w_int = LinearRegression()
lr_no_int = LinearRegression(fit_intercept=False)
neigh = KNeighborsRegressor(n_neighbors=10)
rf = RandomForestRegressor(n_estimators=100)
dt = DecisionTreeRegressor(max_depth = 10)
model_xgb = xgb.XGBRegressor(max_depth=5, n_estimators=1000, learning_rate=0.01)
model_lgb = lgb.LGBMRegressor(learning_rate=0.01, max_depth=5, n_estimators=1000)

score_dt = rmse_cv(dt,n_folds)
print("Decision Tree Regression score: {:.4f} ({:.4f})\n".format(score_dt.mean(), score_dt.std()))

score_rf = rmse_cv(rf,n_folds)
print("Random Forest Regression score: {:.4f} ({:.4f})\n".format(score_rf.mean(), score_rf.std()))

score_xg = rmse_cv(model_xgb,n_folds)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score_xg.mean(), score_xg.std()))

score_lgbm = rmse_cv(model_lgb,n_folds)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score_lgbm.mean(), score_lgbm.std()))